In [1]:
import pandas as pd
import os
import numpy as np
import soundfile as sf
from IPython.display import clear_output
import pickle
import librosa
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, GRU, SimpleRNN
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Wczytanie embeddignów wygenerowanych przez trzy modele

names_of_models = ['LSTM', 'GRU', 'RNN']

all_person_embedding = []

for name in names_of_models:
    
    file_path = '/kaggle/input/rnn-test-embedding/all_person_embedding_'

    embedding_one_model = []
    
    for i in range(0,5):
        one_dataset = file_path + str(i) + '_' + name + '.pkl'

        with open(one_dataset, 'rb') as file:
            data = pickle.load(file)

        embedding_one_model.extend(data)
        
    all_person_embedding.append(embedding_one_model)



In [ ]:
for iterator_for_model in range(0, len(all_person_embedding)):
    
    embedding_all_people = all_person_embedding[iterator_for_model]
    
    #Embedding enrollment ma być stworzony z 40 s nagrania
    emrollment_embedding_all_people = []  # Lista do przechowywania embeddingów enrollment dla wszystkich osób
    test_embedding_all_people = []  # Lista do przechowywania embeddingów testowych dla wszystkich osób
    
    # Pętla przetwarzająca embeddingi dla każdej osoby w embedding_all_people
    for i in range(0, len(embedding_all_people)):
        
        # Inicjalizacja zerowych embeddingów dla enrollment i testowych (o długości 128)
        enrollment_embedding = np.zeros(128)
        test_embedding = np.zeros(128)
        test_embedding_one_person = []  # Lista do przechowywania testowych embeddingów dla jednej osoby
    
        # Zaokrąglenie liczby fragmentów danej osoby do najbliższej wielokrotności 10, aby nagrania testowe miały długość 10 s.
        rounded_len = np.floor_divide(len(embedding_all_people[i]), 10) * 10
        
        for j in range(0, rounded_len):
            
            # Dodawanie embeddingów do enrollment dla pierwszych 40 fragmentów - enrollment ma być zbudowany z 
            # 40 s nagrania
            if j < 40:
                enrollment_embedding += embedding_all_people[i][j][0]
            
            # Po co dziesiątym fragmencie: obliczenie średniego embeddingu testowego i resetowanie `test_embedding`
            # embedding test ma mieć długośc 10 s
            elif (j + 1) % 10 == 0:
                test_embedding = test_embedding / 10
                test_embedding_one_person.append(test_embedding)  # Dodanie średniego embeddingu do listy
                test_embedding = np.zeros(128)  # Reset embeddingu testowego do kolejnych obliczeń
            
            # Dodawanie embeddingów do testowego dla kolejnych fragmentów
            else:
                test_embedding += embedding_all_people[i][j][0]
        
        # Obliczenie średniego embeddingu enrollmentowego (średnia z 40 fragmentów)
        enrollment_embedding = enrollment_embedding / 40
    
        # Dodanie średniego embeddingu enrollment i listy testowych embeddingów do głównych list
        emrollment_embedding_all_people.append(enrollment_embedding)
        test_embedding_all_people.append(test_embedding_one_person)
    
        # Wyświetlanie postępu przetwarzania dla każdej osoby
        print((i + 1) / len(embedding_all_people))
        clear_output(wait=True)

    
    with open('enroll_' + names_of_models[iterator_for_model] + '.pkl', 'wb') as file:
        pickle.dump(emrollment_embedding_all_people, file)

    with open('test_' + names_of_models[iterator_for_model] + '.pkl', 'wb') as file:
        pickle.dump(test_embedding_all_people, file)
        

1.0


In [ ]:
# Wczytuję wyżej policzone embedding enrollment oraz test

enrolls = []
tests = []

enroll_test_list = ['enroll', 'test']

for witch_data in enroll_test_list:
    
    path = '/kaggle/working/' + witch_data + '_'
    
    for name in names_of_models:
        
        one_dataset = path + name + '.pkl'
        
        with open(one_dataset, 'rb') as file:
            data = pickle.load(file)

        if witch_data == 'enroll':
            enrolls.append(data)
        else:
            tests.append(data)

In [ ]:
# W pętli liczone są podobieństwa kosinusowe pomiędzy enrollment a test dla embeddingów wgenerowanych przez każdy model, 
# na podstawie tych danych będą przeprowadzone ewaluacjie modeli

for iterator_for_model in range(0, len(names_of_models)):
    # Iteracyjnie pobierane są kolejne dane odpowiadające kolejnym modelom
    emrollment_embedding_all_people = enrolls[iterator_for_model]
    test_embedding_all_people = tests[iterator_for_model]
    
    # DataFrame do przechowywania wyników
    data_rows = []
    data_rows_trans = []
    
    for i in range(0, len(emrollment_embedding_all_people)):
        embedding_enrollment = emrollment_embedding_all_people[i].reshape(1, -1)
    
        for j in range(0, len(test_embedding_all_people)):
            # Wczytanie embeddingów testowych dla danej osoby
            one_person_list = np.array(test_embedding_all_people[j])
            random_numbers = np.random.uniform(0, 1, len(one_person_list)) > 0.5
            one_person_list = one_person_list[random_numbers]
    
            for h in range(0, len(one_person_list)):
                # Wczytanie embeddingu testowego
                embedding_eval = one_person_list[h].reshape(1, -1)
                # Obliczenie podobieństwa kosinusowego
                cos_sim = cosine_similarity(embedding_enrollment, embedding_eval)[0][0]
                # Zapisanie wyników
                data_rows.append([i, j, i == j, cos_sim])
    
        clear_output(wait=True)
        print(f"Processing ID: {i}")
    
    # Tworzenie DataFrame z listy wyników
    long_data_frame = pd.DataFrame(data_rows, columns=['ID_enrollment', 'ID_test', 'genuine', 'score'])
    long_data_frame.to_csv('long_data_frame' + names_of_models[iterator_for_model] + '.csv', index=False)

Processing ID: 49
